In [1]:
# load in packages
import arcpy
from arcpy.sa import *
from arcpy.ia import *
import gc

In [2]:
# load in slope and make a list of years 
years = list(range(1985, 2024))
slope = arcpy.Raster("other.gdb/slope") # add path

In [ ]:
def annual_lc_stats(year):
    # load raster for year 
    nlcd_raster = arcpy.Raster(f"nlcd.gdb\Annual_NLCD_LndCov_{year}_CU_C1V0_yVWcgPt3Sk5G0VIKGTWu")
    # reclass with runoff coe as int
    run_coe = arcpy.sa.Reclassify(nlcd_raster, "Value", RemapValue([[11, "NODATA"], [12, "NODATA"], [21, 55], [22, 65], [23, 70], 
                                                                    [24, 90],[31, 60], [41, 20], [42, 15], [43, 18],
                                                                    [51, 25], [52, 25], [71, 35], [72, 35], [73, 15],
                                                                    [74, 15], [81, 40], [82, 55], [90, 6], [95, 5]]), "NODATA")

    # land cover stats
    nlcd_table = f".\outputs.gdb\lc_stats_{year}"
    nlcd_results_table = arcpy.ia.SummarizeCategoricalRaster(nlcd_raster, nlcd_table)
    arcpy.conversion.TableToTable(nlcd_results_table, r"C:\Users\clayk\Documents\geog_452_project\out_puts", f"lc_stats_{year}.csv")
    # make int into floats
    run_coe = run_coe * 0.01
    # calc runoff profile with (run_coe * slope) 
    runoff_profile = run_coe * slope

    del run_coe
    # save runoff profile
    runoff_profile.save(f".\outputs.gdb\RunoffPr{year}")
    # get stats for each land cover type, save table, can be used later for more stat analysis
    out_table = f".\outputs.gdb\runoff_stats_{year}"
    results_table = arcpy.sa.ZonalStatisticsAsTable(nlcd_raster, "Value", runoff_profile, out_table)
    arcpy.conversion.TableToTable(results_table, ".\out_puts", f"runoff_stats_{year}.csv")
    # clean env
    del nlcd_raster, runoff_profile, out_table, results_table, nlcd_table, nlcd_results_table
    gc.collect()

    

In [ ]:
# run the analysis over all the years
for year in years:
    annual_lc_stats(year)
    print(year)
    




1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
